# Notebook for setup and use of moving average feature

## Imports and Django Setup

In [1]:
from datetime import timedelta
import math
import os
import pandas as pd
import django
from django.db import connection

from django.utils import timezone
import logging
import numpy as np
from IPython.core.display import display, HTML
os.chdir('..')

# Allows async calls to django ORM in Jupyter. Required.
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

from django_celery_beat import models as cm

from pricedata import models as pd_models
from feature import models as ft_models
from algobuilder.utils import DatabaseUtility

## Add a feature execution for all active datasource symbols
I will be setting up a 1 hour moving average against 1S candle data

In [11]:
# Get all datasource symbols and the feature
active_dss = pd_models.DataSourceSymbol.objects.filter(retrieve_price_data=True)
feature = ft_models.Feature.objects.filter(name='1 Hour Moving Average')[0]

# For each datasource symbol, create a feature exeuction and link the datasoure symbol
for dss in active_dss:
    fex = ft_models.FeatureExecution(feature=feature, name=f'1H-MAvg {dss.symbol.name}')
    fex.save()
    
    fex_dss = ft_models.FeatureExecutionDataSourceSymbol(feature_execution=fex, datasource_symbol=dss, candle_period='1S')
    fex_dss.save()

# Show what we have
ft_models.FeatureExecution.objects.all()

<QuerySet [FeatureExecution(feature=1 Hour Moving Average, name=1H-MAvg GBPUSD, active=True, FeatureExecution(feature=1 Hour Moving Average, name=1H-MAvg USDCHF, active=True, FeatureExecution(feature=1 Hour Moving Average, name=1H-MAvg USDJPY, active=True, FeatureExecution(feature=1 Hour Moving Average, name=1H-MAvg USDCNH, active=True, FeatureExecution(feature=1 Hour Moving Average, name=1H-MAvg USDRUB, active=True, FeatureExecution(feature=1 Hour Moving Average, name=1H-MAvg AUDUSD, active=True, FeatureExecution(feature=1 Hour Moving Average, name=1H-MAvg NZDUSD, active=True, FeatureExecution(feature=1 Hour Moving Average, name=1H-MAvg EURDKK, active=True, FeatureExecution(feature=1 Hour Moving Average, name=1H-MAvg USDCAD, active=True, FeatureExecution(feature=1 Hour Moving Average, name=1H-MAvg USDSEK, active=True, FeatureExecution(feature=1 Hour Moving Average, name=1H-MAvg USDHKD, active=True, FeatureExecution(feature=1 Hour Moving Average, name=1H-MAvg USDSGD, active=True, Featu

# Delete any childless feature executions

In [12]:
fex_list = ft_models.FeatureExecution.objects.all()

for fex in fex_list:
    num_children = len(fex.featureexecutiondatasourcesymbol_set.all())
    if num_children == 0:
        print(f"Deleting childless feature execution. {fex}.")
        fex.delete()

# Reconcile. We should have the same number of feature executions and feature execution datasource symbols.

In [14]:
fex_list = ft_models.FeatureExecution.objects.all()
fex_dss_list = ft_models.FeatureExecutionDataSourceSymbol.objects.all()

# Are they all the same length
print(f"Num feature executions: {len(fex_list)}")
print(f"Num feature execution datasource symbols: {len(fex_dss_list)}")
reconciles = (len(fex_list) == len(fex_dss_list))

reconciles

Num feature executions: 70
Num feature execution datasource symbols: 70


True

# Check if we are getting any results
Number of candles vs number of feature calculations

In [19]:
num_candles = pd_models.Candle.objects.count()
num_fex_results = ft_models.FeatureExecutionResult.objects.count()
print(f"num_candles: {num_candles}. num_feature_calculations: {num_fex_results}. diff: {num_candles - num_fex_results}")

num_candles: 35493414. num_feature_calculations: 35493056. diff: 358


# Plot a graph for one symbol showing price data vs moving average

In [68]:
# Get the datasource symbol
dss = pd_models.DataSourceSymbol.objects.filter(datasource__name='MT5', symbol__name='GBPUSD')[0]

# Get the feature execution
fex_dss = ft_models.FeatureExecutionDataSourceSymbol.objects.filter(datasource_symbol=dss)[0]
fex = fex_dss.feature_execution

# Get the candles and calculations as dataframes and join them by time
#candles = pd.DataFrame(list(pd_models.Candle.objects.filter(datasource_symbol=dss)))
results = pd.DataFrame(list(ft_models.FeatureExecutionResult.objects.filter(feature_execution=fex)))
results

""
